In [1]:
from pymongo import MongoClient
from sklearn.model_selection import train_test_split
from keras.wrappers.scikit_learn import KerasRegressor
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import GaussianNoise
from keras.utils import plot_model
from keras.callbacks import TensorBoard
from keras.utils import vis_utils as vizu
from keras.layers import Dropout
from keras import callbacks
from keras import backend
from scipy.integrate import simps
from scipy.stats import f
from math import sqrt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import f_regression, mutual_info_regression
def knn_baseline_model():
    
    def root_mean_squared_error(y_true, y_pred):
        return backend.sqrt(backend.mean(backend.square(y_pred - y_true), axis=-1))
    
    # create model
    model = Sequential()
    #model.add(GaussianNoise(stddev=0.1, input_shape=(14,)))
    model.add(Dense(7, activation='relu', input_shape=(16,)))
    model.add(Dropout(0.5))
    model.add(GaussianNoise(0.1))
    model.add(Dense(24, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss=root_mean_squared_error, optimizer='adam')
    #model.compile(loss='mse', optimizer='adam')
    return model
def coeff_determination(y_true, y_pred):
    ssres = np.sum(np.square(y_true - y_pred))
    sstot = np.sum(np.square(y_true - np.mean(y_pred)))
    return 1 - ssres / sstot
def REC(y_true , y_pred):
    
    Accuracy = []
    Begin_Range = 0
    End_Range = 1.5
    Interval_Size = 0.01
    
    # List of epsilons
    Epsilon = np.arange(Begin_Range , End_Range , Interval_Size)
    
    # Main Loops
    for i in range(len(Epsilon)):
        count = 0.0
        for j in range(len(y_true)):
            y_true[j]=float(y_true[j])
            y_true[j]
            y_pred[j]
            
            np.linalg.norm(y_true[j] - y_pred[j])
            np.sqrt( np.linalg.norm(y_true[j]) **2 + np.linalg.norm(y_pred[j])**2 ) < Epsilon[i]
            
            if np.linalg.norm(y_true[j] - y_pred[j]) / np.sqrt( np.linalg.norm(y_true[j]) **2 + np.linalg.norm(y_pred[j])**2 ) < Epsilon[i]:
                count = count + 1
        
        Accuracy.append(count/len(y_true))
    
    # Calculating Area Under Curve using Simpson's rule
    AUC = simps(Accuracy , Epsilon ) / End_Range
        
    # returning epsilon , accuracy , area under curve    
    return Epsilon, Accuracy, AUC
def model_evaluate (model, X_train, X_test, y_train, y_test):

    y_pred = model.predict(X_test)
    Deviation, Accuracy, auc = REC(y_test.values, y_pred)
    rmse = sqrt(mean_squared_error(y_test, y_pred))
    #r2 = r2_score(y_test, y_pred)
    r2 = coeff_determination(y_test, y_pred)
    mape = ((y_test - y_pred) / y_test).abs().mean()*100
    
    f_test = np.var(y_test) / np.var(y_pred)
    df1 = len(y_test) - 1
    df2 = len(y_pred) - 1
    p_value = 1 - 2 * abs(0.5 - f.cdf(f_test, df1, df2))
    
    p_stars = ''
    if p_value <= 0.05:
        p_stars = '*'
    if p_value <= 0.01:
        p_stars = '**'
    if p_value <= 0.001:
        p_stars = '***'
    #print(F, p_value)
    
    y_validate = model.predict(X_train)
    training_loss = sqrt(mean_squared_error(y_train, y_validate))
    
    return_dict = {'rmse':rmse, 
                   'mape':mape, 
                   'r2':r2, 
                   'auc':auc,
                   'training_loss':training_loss, 
                   'f_test':f_test,
                   'p':p_stars,
                   'y_pred':y_pred, 
                   'y_validate':y_validate}
    
    return return_dict

C:\Users\user\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
sp103 = pd.read_csv('1031SP_w10.csv')
sp103.fillna(0.0, inplace=True)
sp103 = sp103.drop(['name'], axis=1)
sp103 = sp103.drop(['username'], axis=1)
sp103 = sp103.drop(['final_score'], axis=1)
sp103 = sp103.drop(['label_2'], axis=1)
sp103.head(3)

,video_watching_days,video_watching_times,video_watching_total_time,bbs_days,bbs_num,video_watching_num,video_time_between_start_first_watching,pre_watching_num,in_watching_num,re_watching_num,no_watching_num,post_watching_num,online_quiz,online_hw,pgm,quiz,adjust_score
0,0,0,0,0,0,0,0,0,0,0,28,0,0,0,0,56,7
1,3,6,576,0,0,5,134,0,0,0,28,3,0,0,96,48,62
2,25,185,1846,32,32,91,403,0,15,12,13,11,372,150,95,60,80


In [3]:
data=sp103
data.to_csv('cycu sp103.csv')
data_y= data['adjust_score']
data_x = data.drop(['adjust_score'], axis=1)

In [4]:
data

,video_watching_days,video_watching_times,video_watching_total_time,bbs_days,bbs_num,video_watching_num,video_time_between_start_first_watching,pre_watching_num,in_watching_num,re_watching_num,no_watching_num,post_watching_num,online_quiz,online_hw,pgm,quiz,adjust_score
0,0,0,0,0,0,0,0,0,0,0,28,0,0,0,0,56,7
1,3,6,576,0,0,5,134,0,0,0,28,3,0,0,96,48,62
2,25,185,1846,32,32,91,403,0,15,12,13,11,372,150,95,60,80
3,3,22,490,1,1,9,191,0,0,0,28,3,295,270,99,60,66
4,2,4,1,1,1,2,72,0,0,0,28,1,295,80,0,76,33
5,5,9,54,1,1,4,72,0,0,0,28,3,395,280,98,68,62
6,4,4,0,1,1,0,0,0,0,0,28,0,195,240,87,36,73
7,8,102,934,1,1,45,227,0,0,0,28,19,392,290,93,60,58
8,2,2,0,0,0,0,0,0,0,0,28,0,300,110,70,28,28
9,3,14,438,0,0,11,134,0,0,0,28,8,0,60,0,24,7


In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data_x, data_y, test_size=0.3)

In [6]:
mlp = KerasRegressor(build_fn=knn_baseline_model, epochs=1000, batch_size=2, verbose=0)

In [7]:
mlp.fit(X_train, y_train) 
result_dict = model_evaluate (mlp, X_train, X_test, y_train, y_test)
print(result_dict)

{'rmse': 28.713992521818742, 'mape': 44.61069592253683, 'r2': 0.2868402603524861, 'auc': 0.7650877192982456, 'training_loss': 29.72458843652698, 'f_test': 5.428571112341641, 'p': '***', 'y_pred': array([39.436752, 28.11708 , 11.471223, 27.798077, 20.616856, 17.591974,
       24.657055, 21.174557, 53.83596 , 45.328224, 40.113754, 32.192287,
       46.783382, 38.630806, 40.30766 , 47.27307 , 49.580246, 23.795021,
       31.480038, 45.11575 , 48.898064, 35.245552, 50.905586, 49.90873 ,
       31.401863, 20.567917, 40.21662 , 33.736706, 48.173813, 45.478844,
       41.858242, 43.801117, 33.734966, 40.00579 , 29.783993, 36.26456 ,
       35.880745, 44.551437], dtype=float32), 'y_validate': array([47.39501 , 48.591225, 39.958057, 46.4353  , 39.140034, 45.31433 ,
       36.128387, 42.82517 , 35.69019 , 25.195137, 39.06657 , 37.006565,
       46.342953, 47.30774 , 28.079758, 44.947277, 29.935543, 26.191856,
       24.847538, 36.786476, 44.01069 , 34.58195 , 54.030174, 36.099316,
       37.9345